In [ ]:
import json
import sys
import os

# parent_dir = os.path.abspath(os.getcwd())
# vendor_dir = os.path.join(parent_dir, 'vendor')
# sys.path.append(vendor_dir)

# from tripadvisorCrawler.items import ReviewItem

from items import *
from dataframe import *

import matplotlib
import matplotlib.pyplot as plt
matplotlib.style.use('ggplot')
%matplotlib inline
import numpy as np

import calendar
from ggplot import *

In [ ]:
# data open and auto parse

with open('data/review_items.jl', 'r') as f:
    raw = f.read()

reviews = as_dataframe(raw)

with open('data/hotel_items.jl','r') as f:
    raw = f.read()

hotels = as_dataframe(raw)

In [ ]:
## show case hotels
hotels.head(3)

In [ ]:
# number of hotels
hotels.shape[0]

In [ ]:
# show case reviews
reviews.head(3)

In [ ]:
#summary of rating of dublin hotels
reviews.ix[:,['rating']].describe()

In [ ]:
# reviews['rating_date'] = pd.to_datetime(reviews['rating_date'], format='%Y-%m-%dT%H:%M:%S')
# timeseries = reviews.set_index('rating_date')
timeseries = reviews.set_index('timestamp_rating')

ts2014 = timeseries['2014-01-01':'2014-12-31'].ix[:,'rating'].resample('M').count()
ts2015 = timeseries['2015-01-01':'2015-12-31'].ix[:,'rating'].resample('M').count()
ts2016 = timeseries['2016-01-01':'2016-12-31'].ix[:,'rating'].resample('M').count()

# months = ['Jan','Feb','Mar','Apr','May','Jun','Jul','Aug','Sep','Oct','Nov','Dec']
month_factor = calendar.month_abbr[1:13]
year_factor = ['2014','2015','2016']

xlabel = 'Month'
ylabel = 'Ratings'


# df4 = pd.DataFrame({xlabel:ts2014.index, ylabel:ts2014.values})
df4 = pd.DataFrame({xlabel:month_factor, ylabel:ts2014.values})
df5 = pd.DataFrame({xlabel:month_factor, ylabel:ts2015.values})

pad_x3_index = pd.date_range('1/1/2016',periods=12,freq='M')
# print(pad_x3_index)
ts2016=ts2016.reindex(pad_x3_index).fillna(0)

df6 = pd.DataFrame({xlabel:month_factor, ylabel:ts2016.values})

# new timestamp series
# newts = pd.concat([df1,df2,df3])
# newts.set_index(xlabel,inplace=True)

# http://stackoverflow.com/questions/37596714/compare-multiple-year-data-on-a-single-plot-python
ax4 = df4.plot(x=xlabel,y=ylabel)
ax5 = df5.plot(ax=ax4)
finalPlot = df6.plot(ax=ax5)
finalPlot.legend(['2014','2015','2016'],loc='best')
finalPlot.set_ylabel('Number of Ratings')
finalPlot.set_xlabel('Months')

# http://blog.yhat.com/posts/aggregating-and-plotting-time-series-in-python.html
# custom all...


months_f = pd.Series(np.tile(month_factor,3)).astype('category')
years_f = pd.Series(np.repeat(year_factor,12)).astype('category')

# 我快吐血了， 为了这个自定义的plot
custom = pd.DataFrame({
        'Month':months_f,
        'Ratings':np.array([ts2014.values,ts2015.values,ts2016.values]).flatten(),
        'Year':years_f
    })

# ggplot(aes(x = 'Month', y = 'Ratings'), custom) + geom_line()
# ggplot(aes(x='Month',y='Ratings',colour='Year'),custom) + geom_line()


In [ ]:




# newts['Month'] = map(lambda x: cal.month_abbr[x.month], newts.index)
# newts['Year'] = map(lambda x: str(x.year), newts.index)
# newts['Year'] = newts['Year'].astype('category')

# print('before melt: ')
# print(newts.head(6))

# newts.plot()

# df4 = pd.melt(nots, id_vars=['Month','Year'],value_vars=['rating_count'])

# print("melted: ")
# print(type(df4))
# print(df4.head(3))

# from ggplot import ggplot, ggtitle, geom_line, aes
# p1 = ggplot(aes(x=xlabel, y=ylabel), df1) + geom_line()
# p2 = ggplot(aes(x=xlabel, y=ylabel), df2) + geom_line()
# p3 = ggplot(aes(x=xlabel, y=ylabel), df3) + geom_line()

# p1.show()
# p2.show()
# p3.show()

# ggplot(aes(x='Month', y='rating_count',colour='Year'),nots)+geom_line() + ggtitle("Monthly Rating Count for last 2 years")

In [ ]:
reviewByHotels = reviews.groupby('item_id')

In [ ]:
import pandas as pd
for hotel_url, group in reviewByHotels:
    subset = group.ix[:,['timestamp_rating', 'rating']]
    summary = subset.ix[:,'rating'].describe()
#     print(summary)
#     subset['rating_date'] = pd.to_datetime(subset['rating_date'], format='%Y-%m-%dT%H:%M:%S')
    subset.set_index('timestamp_rating',inplace=True)
    ## Do my own thing: input is subset of rating with timestamp index. output is bin
    ## count for last 2 years of rating on a weekly span.
    
    
    
    
    
#     print(subset.resample('AS').count()) # annual start
#     last2years = pd.date_range('2014-01-01','2016-09-01')
#     last2years = pd.period_range('2014','2016')
#     print(last2years)
#     weeklyCount = subset['2014-01-01':'2016-09-01'].resample('AS').count()
    print(subset['2014-01-01':'2016-09-01'].resample('AS').count())
#     print(weeklyCount)
#     print(subset["2014-01-01":"2016-09-01"])
        
